In [ ]:
# make sure to run this on virtual env with python 3.8.20
! pip install pandas scikit-learn torch essentia

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import essentia.standard as es
import torch.nn.functional as F
from torch.utils.data import Dataset


# creating encoder class to use later
class SongAutoencoder(nn.Module):
    def __init__(self, input_dim=72, hidden_dim=128, bottleneck_dim=128):
        super(SongAutoencoder, self).__init__()

        # Use your existing SongEncoder
        self.encoder = SongEncoder(input_dim, hidden_dim, bottleneck_dim)

        # Add decoder
        self.decoder = nn.Sequential(
            nn.Linear(bottleneck_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Linear(hidden_dim, input_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

def train_autoencoder(autoencoder, dataloader, epochs=50, lr=1e-3, device="cuda" if torch.cuda.is_available() else "cpu"):
    autoencoder = autoencoder.to(device)
    optimizer = torch.optim.Adam(autoencoder.parameters(), lr=lr)
    criterion = nn.MSELoss()

    for epoch in range(epochs):
        total_loss = 0
        for batch in dataloader:
            batch = batch.to(device)
            optimizer.zero_grad()

            reconstructed = autoencoder(batch)
            loss = criterion(reconstructed, batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs} | Loss: {total_loss / len(dataloader):.4f}")

# after i save features from song into json, i grab specific ones to embed
def extract_features_from_json(json_file):

    # open json file
    with open(json_file, "r") as f:
        data = json.load(f)

    # pick relevant features and return numpy array
    features = [

        data["lowlevel"]["average_loudness"],
        data["lowlevel"]["dissonance"]["mean"],
        data["lowlevel"]["dynamic_complexity"],
        data["lowlevel"]["spectral_centroid"]["mean"],
        data["lowlevel"]["spectral_flux"]["mean"],
        data["lowlevel"]["spectral_entropy"]["mean"],
        data["lowlevel"]["spectral_rolloff"]["mean"],
        data["lowlevel"]["hfc"]["mean"],

        data["lowlevel"]["pitch_salience"]["mean"],
        data["lowlevel"]["spectral_complexity"]["mean"],
        data["lowlevel"]["spectral_spread"]["mean"],
        data["lowlevel"]["spectral_strongpeak"]["mean"],

        *data["lowlevel"]["barkbands"]["mean"],
        *data["lowlevel"]["mfcc"]["mean"],
        *data["lowlevel"]["gfcc"]["mean"],

        data["rhythm"]["bpm"],
        data["rhythm"]["danceability"],
        data["rhythm"]["onset_rate"],

        data["tonal"]["chords_strength"]["mean"],
        data["tonal"]["hpcp_crest"]["mean"],
        data["tonal"]["hpcp_entropy"]["mean"],
        data["tonal"]["key_krumhansl"]["strength"]

    ]

    feature_vector = np.array(features)
    return feature_vector

def save_vector_to_csv(vector, file_path):

    # take the vector and save it to a csv file
    df = pd.DataFrame([vector])
    df.to_csv(file_path, index=False, header=False)

    print(f"Feature vector saved to: {file_path}")

def encode_and_save_song_vectors(csv_file, encoder):

    # take the csv file with the vector in it, then encode it
    df = pd.read_csv(csv_file, header=None)
    song_vector = df.values.flatten().astype(np.float32)

    with torch.no_grad():
        song_tensor = torch.tensor(song_vector).view(1, -1) 
        encoded_vector = encoder(song_tensor).squeeze(0).numpy()


    save_vector_to_csv(encoded_vector, csv_file)

def extract_encode(artist, song_name, audio_folder="audio", json_folder="song_features", csv_folder="song_csv", encoder=None): 
    audio_file = os.path.join(audio_folder, f"{artist} - {song_name}.wav")
    json_file = os.path.join(json_folder, f"{artist} - {song_name}.json")
    csv_file = os.path.join(csv_folder, f"{artist} - {song_name}.csv")

    vector_folder = "song_vector"
    vector_csv = os.path.join(vector_folder, f"{artist} - {song_name}.csv")

    features, _ = es.MusicExtractor(
        lowlevelStats=['mean', 'stdev'],
        rhythmStats=['mean', 'stdev'],
        tonalStats=['mean', 'stdev'])(audio_file)

    # features to json -> extract to feature vector -> save to csv
    es.YamlOutput(filename=json_file, format='json')(features)
    feature_vector = extract_features_from_json(json_file)

    save_vector_to_csv(feature_vector, vector_csv)
    save_vector_to_csv(feature_vector, csv_file)

    # encode and save to csv
    if encoder:
        encode_and_save_song_vectors(csv_file, encoder)

def mass_encoder(csv_file, audio_folder="audio", json_folder="song_features", csv_folder="song_csv", encoder=None):
  
    # read csv and check if required columns exist
    pair_data = pd.read_csv(csv_file)
    if not all(col in pair_data.columns for col in ["shortcode", "music_artist", "music_title"]):
        raise ValueError("CSV file must contain 'shortcode', 'music_artist', and 'music_title' columns.")
    
    # filter out rows with nan values in song
    pair_data = pair_data[pair_data["music_artist"].notna() & pair_data["music_title"].notna()]

    output_file = os.path.join(os.path.dirname(csv_file), "pairs_songencoded.csv")
    output_data = []
    
    # tterate through each row in the CSV file
    for index, row in pair_data.iterrows():

        artist = row["music_artist"].replace("'", "")
        song_name = row["music_title"].replace("'", "")
        shortcode = row["shortcode"]
        image_link = row["head_image_url"]

        print(f"Processing song {index + 1}: {artist} - {song_name}")

        try:
            # extract features and encode the song
            extract_encode(
                artist=artist,
                song_name=song_name,
                audio_folder=audio_folder,
                json_folder=json_folder,
                csv_folder=csv_folder,
                encoder=encoder
            )

            song_csv_path = os.path.join(csv_folder, f"{artist} - {song_name}.csv")
            embedding_df = pd.read_csv(song_csv_path, header=None)
            embedding_vector = embedding_df.values.flatten().tolist()

            output_data.append({
                "shortcode" : shortcode,
                "link" : image_link,
                "embedding" : embedding_vector
            })

            pd.DataFrame(output_data).to_csv(output_file, index=False)

        except Exception as e:
            print(f"Failed to process {artist} - {song_name}: {e}")




In [ ]:
if __name__ == "__main__":
    # path to csv
    csv_file = "BARKBARKWOOF_RIBBIT.csv" 

    input_dim = 72
    hidden_dim = 128
    output_dim = 128
    encoder = SongEncoder(input_dim, hidden_dim, output_dim)

    # idk wtf this does lol
    model_path = "song_encoder.pth"
    if os.path.exists(model_path):
        encoder.load_state_dict(torch.load(model_path, weights_only=True))
        encoder.eval()
        print(f"Loaded pre-trained encoder from {model_path}")

    # use goat function and boom boom done (make sure to download songs in spotdl.ipynb first)
    mass_encoder(csv_file=csv_file, encoder=encoder)

    # save the encoder (only for the small model)
    # torch.save(encoder.state_dict(), "song_encoder.pth")

Processing song 1: John Lennon - Beautiful Boy (Darling Boy)
Feature vector saved to: song_vector/John Lennon - Beautiful Boy (Darling Boy).csv
Feature vector saved to: song_csv/John Lennon - Beautiful Boy (Darling Boy).csv
Feature vector saved to: song_csv/John Lennon - Beautiful Boy (Darling Boy).csv
Processing song 2: Pixies - Where Is My Mind? (2007 Remaster)
Failed to process Pixies - Where Is My Mind? (2007 Remaster): In MusicExtractor.compute: MetadataReader: File does not exist or does not seem to be of a supported filetype. AudioLoader: Could not open file "audio/Pixies - Where Is My Mind? (2007 Remaster).wav", error = No such file or directory
Processing song 3: M.I.A. - Bad Girls


[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done


Feature vector saved to: song_vector/M.I.A. - Bad Girls.csv
Feature vector saved to: song_csv/M.I.A. - Bad Girls.csv
Feature vector saved to: song_csv/M.I.A. - Bad Girls.csv
Processing song 4: Soap&Skin - Me and the Devil
Feature vector saved to: song_vector/Soap&Skin - Me and the Devil.csv
Feature vector saved to: song_csv/Soap&Skin - Me and the Devil.csv
Feature vector saved to: song_csv/Soap&Skin - Me and the Devil.csv
Processing song 5: Blood Orange - Champagne Coast


[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done


Feature vector saved to: song_vector/Blood Orange - Champagne Coast.csv
Feature vector saved to: song_csv/Blood Orange - Champagne Coast.csv
Feature vector saved to: song_csv/Blood Orange - Champagne Coast.csv
Processing song 6: Justin Hurwitz - Mia & Sebastian’s Theme
Feature vector saved to: song_vector/Justin Hurwitz - Mia & Sebastian’s Theme.csv
Feature vector saved to: song_csv/Justin Hurwitz - Mia & Sebastian’s Theme.csv
Feature vector saved to: song_csv/Justin Hurwitz - Mia & Sebastian’s Theme.csv
Processing song 8: Empire Of The Sun - We Are The People


...

[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done


Feature vector saved to: song_vector/The Beatles - Here Comes The Sun.csv
Feature vector saved to: song_csv/The Beatles - Here Comes The Sun.csv
Feature vector saved to: song_csv/The Beatles - Here Comes The Sun.csv
Processing song 80: Elton John - Goodbye Yellow Brick Road


[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done


Feature vector saved to: song_vector/Elton John - Goodbye Yellow Brick Road.csv
Feature vector saved to: song_csv/Elton John - Goodbye Yellow Brick Road.csv
Feature vector saved to: song_csv/Elton John - Goodbye Yellow Brick Road.csv
Processing song 81: The Clash - London Calling
Feature vector saved to: song_vector/The Clash - London Calling.csv
Feature vector saved to: song_csv/The Clash - London Calling.csv
Feature vector saved to: song_csv/The Clash - London Calling.csv


[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
